<a href="https://colab.research.google.com/github/hukim1112/MLDL/blob/master/object_detection/data_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 다운로드

### https://drive.google.com/drive/folders/1z-FSY3JN-FWBl87XAZyb4m2KcHqJEUTm?usp=sharing

### Colab 사용 시 GDUT_HWD.json과 images.zip을 Colab repository에 업로드 해주세요.

In [ ]:
#데이터셋 업로드 후 압축 풀 것.
!unzip images.zip

In [ ]:
import os
import cv2
from matplotlib import pyplot as plt

image_dir = '/content/images'
annotation = '/content/GDUT_HWD.json'

#압축
img_list = os.listdir(image_dir)
img_path = os.path.join(image_dir, img_list[2])
img = cv2.imread(img_path)[:,:,::-1]
plt.imshow(img)

# COCO library
데모 : https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

coco 데이터셋 형식의 레이블을 로드하고, 읽을 수 있습니다.


In [ ]:
from pycocotools.coco import COCO

coco_obj = COCO("/content/GDUT_HWD.json")

In [ ]:
# 로드 가능한 이미지의 id들
coco_obj.getImgIds()[:10]

In [ ]:
# 특정 이미지 id의 레이블 정보를 검색
ann_ids = coco_obj.getAnnIds(imgIds=1, catIds=[1,2], iscrowd=None)

In [ ]:
ann_ids

In [ ]:
for ann_id in ann_ids:
  x, y, w, h = coco_obj.loadAnns(ann_id)[0]['bbox']
  print("정답 박스 정보", x, y, w, h)
  print("클래스 정보", coco_obj.loadAnns(ann_id)[0]['category_id'])

# 텐서플로우 데이터 오브젝트 생성

data.py의 Dataset 클래스는 tf.data.Dataset.from_generator 함수를 사용해 파이썬 iterator로부터 텐서플로우 데이터셋 오브젝트를 만듭니다. 이런 방식을 파일을 준비할 때 tensorflow 이외의 라이브러리(ex. opencv나 coco library)를 함께 사용해야 할 때 유용합니다.

우선 프로젝트 가이드에서 data.py와 anchor.py 그리고 box_utils.py를 가져와 import 합니다.

https://drive.google.com/drive/folders/16ccs8882-jswD72J4k4L1jod3nTfCpni?usp=sharing

In [ ]:
import data
from anchor import generate_default_boxes

In [ ]:
# anchor box 생성
anchor_param = {"ratios": [[2], [2, 3], [2, 3], [2, 3], [2]],
                           "scales": [0.1, 0.3, 0.5, 0.7, 0.9, 1.075],
                           "fm_sizes": [28, 14, 7, 3, 1],
                           "image_size": 224} #anchor parameters
anchors = generate_default_boxes(anchor_param)

In [ ]:
label_set = ["head","helmet"]
input_shape = (224,224,3)
image_dir = '/content/images'
annotation = '/content/GDUT_HWD.json'

In [ ]:
ds_obj = data.Dataset(COCO, anchors, image_dir, annotation, label_set, input_shape)

In [ ]:
train_ds, train_length = ds_obj.load_data_generator('train')
print("Dataset length : ", train_length)

In [ ]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(train_ds.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)

# 데이터 파이프라인 구축
셔플, 미니배치, prefetch 연결

In [ ]:
import tensorflow as tf

batch_size = 16
shuffle_buffer = 159

train_ds = train_ds.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for i, (_, imgs, gt_confs, gt_locs) in enumerate(train_ds.take(1)):
    print(imgs.shape, gt_confs.shape, gt_locs.shape)